In [1]:
%cd /content
# remove first if exists
# !rm -rf arabic-poetry-speech-classification
# !git clone https://github.com/MagedSaeed/arabic-poetry-speech-classification.git

/content


In [2]:
# mount the drive to get the datasets
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [3]:
# put the task path you want to work on here
# %cd arabic-poetry-speech-classification/fine-tuning

In [4]:
!pip install -q torchaudio
!pip install -q transformers
!pip install -q datasets
!pip install -q lang_trans
!pip install -q arabic_reshaper
!pip install -q python-bidi
!pip install -q pydub
!pip install -q soundfile
!pip install -q jiwer
!pip install -q PyArabic

     |████████████████████████████████| 5.3 MB 5.0 MB/s 
     |████████████████████████████████| 7.6 MB 65.3 MB/s 
     |████████████████████████████████| 163 kB 91.1 MB/s 
     |████████████████████████████████| 441 kB 5.1 MB/s 
     |████████████████████████████████| 115 kB 85.3 MB/s 
     |████████████████████████████████| 212 kB 93.3 MB/s 
     |████████████████████████████████| 127 kB 91.6 MB/s 
     |████████████████████████████████| 1.4 MB 5.0 MB/s 
     |████████████████████████████████| 2.2 MB 91.3 MB/s 
     |████████████████████████████████| 126 kB 5.1 MB/s 


In [1]:
import transformers
from transformers import HfArgumentParser,TrainingArguments
from transformers.trainer_utils import get_last_checkpoint, is_main_process
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2Model,Wav2Vec2PreTrainedModel,Trainer,Wav2Vec2FeatureExtractor

In [2]:
import os
import re
import sys
import json
import torch
import jiwer
import logging
import librosa
import datasets
import itertools
import torchaudio
import numpy as np
import pandas as pd
from torch import nn
import seaborn as sns
from tqdm import tqdm
import soundfile as sf
import arabic_reshaper
from pyarabic import araby
from pydub import AudioSegment
import matplotlib.pyplot as plt
from pydub.utils import mediainfo
from argparse import ArgumentParser
from collections import defaultdict
from torch.nn import functional as F
from contextlib import contextmanager
from bidi.algorithm import get_display
from lang_trans.arabic import buckwalter
from dataclasses import dataclass, field
from datasets import load_dataset, Dataset
from sklearn.metrics import accuracy_score
from typing import Any, Dict, List, Optional, Union
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
%matplotlib inline

In [3]:
class CTCTrainer(Trainer):
    def training_step(
        self, model: nn.Module, inputs: Dict[str, Union[torch.Tensor, Any]]
    ) -> torch.Tensor:
    
        model.train()
        inputs = self._prepare_inputs(inputs)

        if self.use_amp:
            with autocast():
                loss = self.compute_loss(model, inputs)
        else:
            loss = self.compute_loss(model, inputs)

        if self.args.n_gpu > 1:
            if model.module.config.ctc_loss_reduction == "mean":
                loss = loss.mean()
            elif model.module.config.ctc_loss_reduction == "sum":
                loss = loss.sum() / (inputs["labels"] >= 0).sum()
            else:
                raise ValueError(
                    f"{model.config.ctc_loss_reduction} is not valid. Choose one of ['mean', 'sum']"
                )

        if self.args.gradient_accumulation_steps > 1:
            loss = loss / self.args.gradient_accumulation_steps

        if self.use_amp:
            self.scaler.scale(loss).backward()
        elif self.use_apex:
            with amp.scale_loss(loss, self.optimizer) as scaled_loss:
                scaled_loss.backward()
        elif self.deepspeed:
            self.deepspeed.backward(loss)
        else:
            loss.backward()

        return loss.detach()


In [4]:
class CustomWav2Vec2Processor:

    def __init__(self, feature_extractor):

        self.feature_extractor = feature_extractor
        self.current_processor = self.feature_extractor

    def save_pretrained(self, save_directory):
        
        self.feature_extractor.save_pretrained(save_directory)

    @classmethod
    def from_pretrained(cls, pretrained_model_name_or_path, **kwargs):
       
        feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(
            pretrained_model_name_or_path, **kwargs
        )

        return cls(feature_extractor=feature_extractor)

    def __call__(self, *args, **kwargs):
       
        return self.current_processor(*args, **kwargs)

    def pad(self, *args, **kwargs):
        
        return self.current_processor.pad(*args, **kwargs)


In [5]:
def list_field(default=None, metadata=None):
    return field(default_factory=lambda: default, metadata=metadata)

@dataclass
class ModelArguments:


    model_name_or_path: str = field(
        metadata={
            "help": "Path to pretrained model or model identifier from huggingface.co/models"
        }
    )
    cache_dir: Optional[str] = field(
        default=None,
        metadata={
            "help": "Where do you want to store the pretrained models downloaded from huggingface.co"
        },
    )
    freeze_feature_extractor: Optional[bool] = field(
        default=True,
        metadata={
            "help": "Whether to freeze the feature extractor layers of the model."
        },
    )
    attention_dropout: Optional[float] = field(
        default=0.1,
        metadata={"help": "The dropout ratio for the attention probabilities."},
    )
    activation_dropout: Optional[float] = field(
        default=0.1,
        metadata={
            "help": "The dropout ratio for activations inside the fully connected layer."
        },
    )
    hidden_dropout: Optional[float] = field(
        default=0.1,
        metadata={
            "help": "The dropout probabilitiy for all fully connected layers in the embeddings, encoder, and pooler."
        },
    )
    feat_proj_dropout: Optional[float] = field(
        default=0.1,
        metadata={
            "help": "The dropout probabilitiy for all 1D convolutional layers in feature extractor."
        },
    )
    mask_time_prob: Optional[float] = field(
        default=0.05,
        metadata={
            "help": "Propability of each feature vector along the time axis to be chosen as the start of the vector"
            "span to be masked. Approximately ``mask_time_prob * sequence_length // mask_time_length`` feature"
            "vectors will be masked along the time axis. This is only relevant if ``apply_spec_augment is True``."
        },
    )
    # gradient_checkpointing: Optional[bool] = field(
    #     default=True,
    #     metadata={
    #         "help": "If True, use gradient checkpointing to save memory at the expense of slower backward pass."
    #     },
    # )
    layerdrop: Optional[float] = field(
        default=0.0, metadata={"help": "The LayerDrop probability."}
    )


@dataclass
class DataTrainingArguments:
    dataset_config_name: Optional[str] = field(
        default=None,
        metadata={
            "help": "The configuration name of the dataset to use (via the datasets library)."
        },
    )
    train_split_name: Optional[str] = field(
        default="train+validation",
        metadata={
            "help": "The name of the training data set split to use (via the datasets library). Defaults to 'train'"
        },
    )
    overwrite_cache: bool = field(
        default=False,
        metadata={"help": "Overwrite the cached preprocessed datasets or not."},
    )
    preprocessing_num_workers: Optional[int] = field(
        default=None,
        metadata={"help": "The number of processes to use for the preprocessing."},
    )
    max_train_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of training examples to this "
            "value if set."
        },
    )
    max_val_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of validation examples to this "
            "value if set."
        },
    )
    chars_to_ignore: List[str] = list_field(
        default=[",", "?", ".", "!", "-", ";", ":", '""', "%", "'", '"', "�"],
        metadata={"help": "A list of characters to remove from the transcripts."},
    )


@dataclass
class DataCollatorCTCWithPadding:

    processor: CustomWav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = 320000
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = 320000
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(
        self, features: List[Dict[str, Union[List[int], torch.Tensor]]]
    ) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods

        input_features = [
            {"input_values": feature["input_values"]} for feature in features
        ]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(
            labels_batch.attention_mask.ne(1), -100
        )

        batch["labels"] = labels
        return batch

## Getting the dataset

In [10]:
!cp -r /content/drive/MyDrive/KFUPM-Master/ICS606/Dataset/dataset_wav.zip .

In [11]:
if os.path.exists('dataset_wav'):
  if len(os.listdir('dataset_wav')) == 0:
    os.system('unzip dataset_wav.zip')
else:
  os.system('unzip dataset_wav.zip')

In [6]:
metadata_path = '/content/drive/MyDrive/KFUPM-Master/ICS606/Dataset/metadata.xlsx'
test_metadata_path = '/content/drive/MyDrive/KFUPM-Master/ICS606/Dataset/test_metadata.xlsx'

In [7]:
!cp $metadata_path .
!cp $test_metadata_path .

In [8]:
dataset_wav_folder = 'dataset_wav'

In [9]:
def preprocess_metadata(metadata):
  metadata['Bahr'] = metadata['Bahr'].apply(lambda bahr: bahr.strip() if isinstance(bahr,str) else bahr)
  metadata['Clean Script'] = metadata['Script'].apply(lambda script: araby.strip_tashkeel(araby.strip_tatweel(script)))
  metadata['File Exist'] = metadata['Utterance name'].apply(lambda file_name: os.path.isfile(f'{dataset_wav_folder}/{file_name}'))
  metadata = metadata[metadata['File Exist'] == True]
  return metadata

In [10]:
metadata = preprocess_metadata(pd.read_excel(metadata_path))
test_metadata = preprocess_metadata(pd.read_excel(test_metadata_path))
train_metadata = metadata[[script not in test_metadata['No.'].values for script in metadata['No.']]]
train_metadata

,No.,Utterance name,Script,Bahr,Source,Name when extacted,Original name in document,Is_Malformed,Reciter_Label,Reciter_Gender,Further info on Bahr,Name of document,Clean Script,File Exist
0,1.0,P-1.aac,شتَّ شَعبُ الحيِّ بعدَ التئامْ وشجاكَ الر...,المديد,ديوان الطرماح ص227,1.aac,1.aac,0.0,0.0,0.0,NaN,بحر المديد,شت شعب الحي بعد التئام وشجاك الربع ربع ال...,True
1,2.0,P-2.aac,حَسَرَتْ عَنْهُ الرِّيَاحُ فَأَبْدَتْ من...,المديد,ديوان الطرماح ص227,2.aac,2.aac,0.0,0.0,0.0,NaN,NaN,حسرت عنه الرياح فأبدت منتأى كالقرو رهن ا...,True
2,3.0,P-3.aac,وخصيفَ اللَّونِ جادَتْ بهِ مَرْخَةٌ مِنْ ...,المديد,ديوان الطرماح ص227,3.aac,3.aac,0.0,0.0,0.0,NaN,NaN,وخصيف اللون جادت به مرخة من مخدج أو تمام,True
3,4.0,P-4.aac,بَيْنَ أظْآرٍ بِمَظْلُومَةٍ كَسَرَاةِ الس...,المديد,ديوان الطرماح ص227,4.aac,4.aac,0.0,0.0,0.0,NaN,NaN,بين أظآر بمظلومة كسراة الساق ساق الحمام,True
4,5.0,P-5.aac,مَنْزِلًا كَانَ لَنَا مَرَّةً وطنًا نحتلّ...,المديد,ديوان الطرماح ص227,5.aac,5.aac,0.0,0.0,0.0,NaN,NaN,منزلا كان لنا مرة وطنا نحتله كل عام,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3681,3731.0,P-3731.mp3,وَلِّ السَلاطينَ مَن تَوَلّاها وَاِلجَأ إ...,المنسرح,-,45.mp3,45.mp3,NaN,NaN,NaN,NaN,NaN,ول السلاطين من تولاها والجأ إليه تكن حدياها,True
3682,3732.0,P-3732.mp3,وَلا تَغُرَّنَّكَ الإِمارَةُ في غَيرِ أَم...,المنسرح,-,46.mp3,46.mp3,NaN,NaN,NaN,NaN,NaN,ولا تغرنك الإمارة في غير أمير وإن بها باهى,True
3683,3733.0,P-3733.mp3,فَإِنَّما المَلكُ رَبُّ مَملَكَةٍ قَد فَغ...,المنسرح,-,47.mp3,47.mp3,NaN,NaN,NaN,NaN,NaN,فإنما الملك رب مملكة قد فغم الخافقين سرياها,True
3684,3734.0,P-3734.mp3,مُبتَسِمٌ وَالوُجوهُ عابِسَةٌ سِلمُ العِد...,المنسرح,-,48.mp3,48.mp3,NaN,NaN,NaN,NaN,NaN,مبتسم والوجوه عابسة سلم العدى عنده كهيجاها,True


In [11]:
test_metadata

,No.,Utterance name,Script,Bahr,Source,Name when extacted,Original name in document,Further info on Bahr,Name of document,Clean Script,File Exist
0,89.0,P-89.aac,إنَّ بالشِّعبِ الَّذي دُونَ سَلْعٍ لَقتيل...,المديد,ديوان ثابت بن جابرص35,89.aac,89.aac,NaN,NaN,إن بالشعب الذي دون سلع لقتيلا دمه ما يطل,True
1,90.0,P-90.aac,خَلَّفَ الْعِبْءَ عَلَيَّ وَولَّى أنا بال...,المديد,ديوان ثابت بن جابرص35,90.aac,90.aac,NaN,NaN,خلف العبء علي وولى أنا بالعبء له مستقل,True
2,91.0,P-91.aac,ووراءَ الثَّأر منِّي ابنُ أختٍ مَصِعٌ عُق...,المديد,ديوان ثابت بن جابرص35,91.aac,91.aac,NaN,NaN,ووراء الثأر مني ابن أخت مصع عقدته ما تحل,True
3,92.0,P-92.aac,مُطْرِقٌ يَرْشَحُ سَمًّا كَمَا أَطْ رَقَ ...,المديد,ديوان ثابت بن جابرص35,92.aac,92.aac,NaN,NaN,مطرق يرشح سما كما أط رق أفعى ينفث السم صل,True
4,93.0,P-93.aac,خبرٌ ما نابَنا مُصْمَئِلٌّ جلَّ حتّى دقَّ...,المديد,ديوان ثابت بن جابرص35,93.aac,93.aac,NaN,NaN,خبر ما نابنا مصمئل جل حتى دق فيه الأجل,True
...,...,...,...,...,...,...,...,...,...,...,...
354,3682.0,P-3682.m4a,حِسانُ الوُجوهِ حِدادُ السُيو فِ يَبتَدِر...,المتقارب,ديوان قيس بن الخطيم ص 73,ÇáÊÓÌíá (98).m4a,التسجيل (98).m4a,NaN,NaN,حسان الوجوه حداد السيو ف يبتدر المجد شبانها,True
355,3683.0,P-3683.m4a,وَبِالشَوطِ مِن يَثرِبٍ أَعبُدٌ سَتَهلِكُ...,المتقارب,ديوان قيس بن الخطيم ص 73,ÇáÊÓÌíá (99).m4a,التسجيل (99).m4a,NaN,NaN,وبالشوط من يثرب أعبد ستهلك في الخمر أثمانها,True
356,3684.0,P-3684.m4a,يَهونُ عَلى الأَوسِ أَثمانُهُم إِذا راحَ ...,المتقارب,ديوان قيس بن الخطيم ص 73,ÇáÊÓÌíá (100).m4a,التسجيل (100).m4a,NaN,NaN,يهون على الأوس أثمانهم إذا راح يخطر نشوانها,True
357,3685.0,P-3685.m4a,أَتَتهُم عَرانينُ مِن مالِكٍ سِراعٌ إِلى ...,المتقارب,ديوان قيس بن الخطيم ص 73,ÇáÊÓÌíá (101).m4a,التسجيل (101).m4a,NaN,NaN,أتتهم عرانين من مالك سراع إلى الروع فتيانها,True


In [12]:
train_metadata,val_metadata = train_test_split(train_metadata,test_size=0.1,random_state=42)
train_metadata

,No.,Utterance name,Script,Bahr,Source,Name when extacted,Original name in document,Is_Malformed,Reciter_Label,Reciter_Gender,Further info on Bahr,Name of document,Clean Script,File Exist
0,1.0,P-1.aac,شتَّ شَعبُ الحيِّ بعدَ التئامْ وشجاكَ الر...,المديد,ديوان الطرماح ص227,1.aac,1.aac,0.0,0.0,0.0,NaN,بحر المديد,شت شعب الحي بعد التئام وشجاك الربع ربع ال...,True
2760,2794.0,P-2794.wma,لنا في كلّ يومٍ منْ معدٍّ سِبابٌ أوْ قِتَ...,الوافر,ديوان حسان بن ثابت دار الكتب العلمية بيروت لبنان,20.wma,20.wma,NaN,NaN,NaN,NaN,NaN,لنا في كل يوم من معد سباب أو قتال أو هجاء,True
1140,1146.0,P-1146.m4a,إن الدنيا قد غرتنا واستهوتنا واستلهتنا,المتدارك,أهدى سبيل إلى علمي الخليل (ص: 73),ÇáÊÓÌíá (2).m4a,التسجيل (2).m4a,0.0,13.0,0.0,NaN,NaN,إن الدنيا قد غرتنا واستهوتنا واستلهتنا,True
3538,3588.0,P-3588.m4a,وَهوَ يَرميها وَلَن يَبلُغَها رَعَةَ الجا...,الرمل,ديوان سويد اليشكري 33,ÇáÊÓÌíá.m4a,التسجيل.m4a,NaN,NaN,NaN,NaN,NaN,وهو يرميها ولن يبلغها رعة الجاهل يرضى ما صنع,True
2565,2599.0,P-2599.aac,زعمتَ أنّك عمّي كذبتَ بلْ أنتَ غمّي,المجتث,المجموع اللفيف (ص: 324),34.aac,34.aac,NaN,NaN,NaN,NaN,NaN,زعمت أنك عمي كذبت بل أنت غمي,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1316,1322.0,P-1322.wma,هُوَ الكاسِي مِنَ السُؤدُ دِ وَالعارِي مِ...,الهزج,NaN,åæ ÇáßÇÓí.wma,هو الكاسي.wma,0.0,9.0,1.0,NaN,NaN,هو الكاسي من السؤد د والعاري من الكبر,True
1351,1357.0,P-1357.wma,وَيا لِلْوَرْدِ ما أَبْها هُ مَنْظُورًا و...,الهزج,NaN,æíÇááæÑÏ.wma,ويا للورد.wma,0.0,9.0,1.0,NaN,NaN,ويا للورد ما أبها ه منظورا ومخبورا,True
1515,1521.0,P-1521.m4a,وَأَضرِبُ بِالسَيفِ يَومَ الوَغى أَقُدُّ ...,الطويل,ديوان عامر بن الطفيل ص120,ÇáÊÓÌíá (41).m4a,التسجيل (41).m4a,0.0,16.0,0.0,NaN,NaN,وأضرب بالسيف يوم الوغى أقد به حلق المبرم,True
1016,1021.0,P-1021.m4a,وَكانَ اِتِّياماً كُلُّ حَرفٍ غَزيرَةٍ أَ...,الطويل,ديوان الأفوه الأودي ص 80,ÇáÊÓÌíá (97).m4a,التسجيل (97).m4a,0.0,11.0,0.0,NaN,NaN,وكان اتياما كل حرف غزيرة أهانوا لها الأمو...,True


In [13]:
val_metadata

,No.,Utterance name,Script,Bahr,Source,Name when extacted,Original name in document,Is_Malformed,Reciter_Label,Reciter_Gender,Further info on Bahr,Name of document,Clean Script,File Exist
52,53.0,P-53.aac,فَتَلاَفَتْهُ، فَلاَثَتْ بِهِ لَعْوَةٌ تَ...,المديد,ديوان الطرماح ص237,53.aac,53.aac,0.0,0.0,0.0,NaN,NaN,فتلافته، فلاثت به لعوة تضبح ضبح النهام,True
746,748.0,P-748.wma,آنَسْتُ في الحَيِّ نارًا لَيْلًا فَبَشَّر...,المجتث,NaN,ÂäÓÊõ.wma,آنستُ.wma,0.0,9.0,0.0,NaN,NaN,آنست في الحي نارا ليلا فبشرت أهلي,True
1597,1603.0,P-1603.m4a,وَإِن تَنقُضي العَهدَ الَّذي كانَ بَينَنا ...,الطويل,ديوان أبي الأسود الدؤلي ص131,ÇáÊÓÌíá (11).m4a,التسجيل (11).m4a,0.0,17.0,0.0,NaN,NaN,وإن تنقضي العهد الذي كان بيننا وتبدي بباق...,True
1458,1464.0,P-1464.m4a,نامَ الخلِيُّ وبتُّ اللّيلَ مُرْتَفِقَا أ...,البسيط,الأعشى الكبير,ÇáÊÓÌíá (93).m4a,التسجيل (93).m4a,0.0,15.0,0.0,NaN,NaN,نام الخلي وبت الليل مرتفقا أرعى النجوم عم...,True
261,262.0,P-262.m4a,إِنَّ العَطيَّةَ خَيرُ ما وَجَّهتَها وَحَ...,الكامل,ديوان أبي الأسود الدؤلي ص51,ÇáÊÓÌíá (2).m4a,التسجيل (2).m4a,0.0,0.0,0.0,NaN,NaN,إن العطية خير ما وجهتها وحسبتها حمدا وأجر...,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2709,2743.0,P-2743.opus,إِنَّ النُّجُوْمَ الَّتِي تُضِيْءُ لنا أَ...,المنسرح,الدر الفريد وبيت القصيد (4/ 463),88.opus,88.opus,NaN,NaN,NaN,NaN,NaN,إن النجوم التي تضيء لنا أصغرها في العيون ...,True
3245,3279.0,P-3279.wma,كَأَنَّ أَوْبَ ذِرَاعَيْهَا إِذَا عَرِقَتْ ...,البسيط,ديوان كعب بن زهير،دارالكتب العلمية،بيروت,ÈÇäÊ 27.wma,بانت 27.wma,NaN,NaN,NaN,NaN,NaN,كأن أوب ذراعيها إذا عرقت وقد تلفع بالقور ...,True
2014,2048.0,P-2048.aac,تَرَكَ الناسُ لَنا أَكتافَهُم وَتَوَلَّوا...,الرمل,ديوان الأفوه الأودي ص77,26.aac,26.aac,NaN,NaN,NaN,NaN,NaN,ترك الناس لنا أكتافهم وتولوا لات لم يغن ا...,True
453,454.0,P-454.wav,وَمَأْكَمةً يَضِيقُ الْبَابُ عَنْها وَكَش...,الوافر,ديوان عمرو بن كلثوم برواية الأصمعي,ãÚáÞÉ_ÚãÑæ_Èä_ßáËæã-021.wav,معلقة_عمرو_بن_كلثوم-021.wav,0.0,6.0,0.0,NaN,معلقة عمرو بن كلثوم- الوافر- 21-30,ومأكمة يضيق الباب عنها وكشحا قد جننت به ج...,True


In [14]:
# sample_rates = set()
# for file_path in tqdm(
#     list(
#         itertools.chain(
#           train_metadata['Utterance name'],
#           test_metadata['Utterance name'],
#           val_metadata['Utterance name']
#       )
#     )
#   ):
#   complete_path = f'{dataset_wav_folder}/{file_path}'
#   complete_wav_path = f'{dataset_wav_folder}/{file_path}'
#   # os.system(f'ffmpeg -i {complete_path} {complete_wav_path}')
#   audio = AudioSegment.from_file(complete_path)
#   sample_rates.add(audio.frame_rate)
#   audio.export(f'{dataset_wav_folder}/{file_path}', format='wav')
# sample_rates

## getting training arguments

In [15]:
# testing path
# --output_dir=sinai-fine-tuned
args_string="""
--model_name_or_path="bakrianoo/sinai-voice-ar-stt"
--freeze_feature_extractor
--output_dir=/content/drive/MyDrive/KFUPM-Master/ICS606/Models/SinaiFineTuned
--num_train_epochs="10" 
--per_device_train_batch_size="2"
--gradient_accumulation_steps="8"
--learning_rate="5e-4" 
--warmup_steps="300" 
--evaluation_strategy="steps"
--save_steps="300" 
--eval_steps="200"
--fp16 \
--preprocessing_num_workers="$(nproc)" 
--save_total_limit="1" 
--logging_steps="10" 
--do_eval=1
--do_train=1
"""
def match_numeric_type(s):
  try:
    return json.loads(s)
  except:
    return s

# construct the args dict from the args string
args_dict = dict()
for line in args_string.splitlines():
  line = line.replace('--','').replace(r'\\','').replace('"','').replace("'",'').strip()
  if line:
    if '=' in line:
      arg_name,arg_value = line.split('=')
      args_dict[arg_name] = arg_value
    else:
      args_dict[line] = ''

# match args to dataclasses
args_class_attrs = defaultdict(list)
for arg_name,arg_value in args_dict.items():
  for ArgClass in (ModelArguments, DataTrainingArguments, TrainingArguments):
    if arg_name in ArgClass.__annotations__:
      args_class_attrs[ArgClass].append(arg_name)

# initialzie args objects
model_args = ModelArguments(**{arg_name:match_numeric_type(args_dict[arg_name]) for arg_name in args_class_attrs[ModelArguments]})
data_args = DataTrainingArguments(**{arg_name:match_numeric_type(args_dict[arg_name]) for arg_name in args_class_attrs[DataTrainingArguments]})
training_args = TrainingArguments(**{arg_name:match_numeric_type(args_dict[arg_name]) for arg_name in args_class_attrs[TrainingArguments]})

## logging setup

In [16]:
# Setup logging
# logging.basicConfig(
#     format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
#     datefmt="%m/%d/%Y %H:%M:%S",
#     handlers=[logging.StreamHandler(sys.stdout)],
# )
# # logger.setLevel(
# #     logging.INFO if is_main_process(training_args.local_rank) else logging.WARN
# # )
# logger.setLevel(
#     logging.WARN
# )
# # Log on each process the small summary:
# logger.warning(
#     f"Process rank: {training_args.local_rank}, device: {training_args.device}, n_gpu: {training_args.n_gpu}"
#     + f"distributed training: {bool(training_args.local_rank != -1)}, 16-bits training: {training_args.fp16}"
# )
# # Set the verbosity to info of the Transformers logger (on main process only):
# if is_main_process(training_args.local_rank):
#     transformers.utils.logging.set_verbosity_info()

# # Set seed before initializing model.
# set_seed(training_args.seed)


## dataset processing

In [17]:
# train_metadata = pd.read_csv(metadata_train_path)
# val_metadata = pd.read_csv(metadata_val_path)
# test_metadata = pd.read_csv(metadata_test_path)

train_dataset = Dataset.from_pandas(train_metadata)
eval_dataset = Dataset.from_pandas(val_metadata)
test_dataset = Dataset.from_pandas(test_metadata)

In [18]:
if data_args.max_train_samples is not None:
    train_dataset = train_dataset.select(range(data_args.max_train_samples))

if data_args.max_val_samples is not None:
    eval_dataset = eval_dataset.select(range(data_args.max_val_samples))
# Preprocessing the datasets.
# We need to read the aduio files as arrays and tokenize the targets.
resamplers = {  # The dataset contains all the uncommented sample rates
    48000: torchaudio.transforms.Resample(48000, 16000),
    44100: torchaudio.transforms.Resample(44100, 16000),
    # 32000: torchaudio.transforms.Resample(32000, 16000),
}

def speech_file_to_array_fn(batch):
    start = 0
    stop = 20
    srate = 16_000
    speech_array, sampling_rate = torchaudio.load(
        f'dataset_wav/{batch["Utterance name"]}'
    )
    speech_array = speech_array[0]
    batch["speech"] = resamplers[sampling_rate](speech_array).squeeze().numpy()
    batch["sampling_rate"] = srate
    batch["target_text"] = batch["Clean Script"]
    return batch

train_dataset = train_dataset.map(
    speech_file_to_array_fn,
    remove_columns=train_dataset.column_names,
    num_proc=data_args.preprocessing_num_workers,
)
eval_dataset = eval_dataset.map(
    speech_file_to_array_fn,
    remove_columns=eval_dataset.column_names,
    num_proc=data_args.preprocessing_num_workers,
)

test_dataset = test_dataset.map(
    speech_file_to_array_fn,
    remove_columns=test_dataset.column_names,
    num_proc=data_args.preprocessing_num_workers,

)

  0%|          | 0/2978 [00:00<?, ?ex/s]

  0%|          | 0/331 [00:00<?, ?ex/s]

  0%|          | 0/359 [00:00<?, ?ex/s]

In [19]:
processor = Wav2Vec2Processor.from_pretrained("bakrianoo/sinai-voice-ar-stt")

def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."
    batch["input_values"] = processor(
        batch["speech"], sampling_rate=batch["sampling_rate"][0]
    ).input_values
    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

train_dataset = train_dataset.map(
    prepare_dataset,
    remove_columns=train_dataset.column_names,
    batch_size=training_args.per_device_train_batch_size,
    batched=True,
    num_proc=data_args.preprocessing_num_workers,
)
eval_dataset = eval_dataset.map(
    prepare_dataset,
    remove_columns=eval_dataset.column_names,
    batch_size=training_args.per_device_train_batch_size,
    batched=True,
    num_proc=data_args.preprocessing_num_workers,
)
test_dataset = test_dataset.map(
    prepare_dataset,
    remove_columns=test_dataset.column_names,
    batch_size=training_args.per_device_train_batch_size,
    batched=True,
    num_proc=data_args.preprocessing_num_workers,
)

  0%|          | 0/1489 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:162: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  "`as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your "


  0%|          | 0/166 [00:00<?, ?ba/s]

  0%|          | 0/180 [00:00<?, ?ba/s]

## check for previous checkpoints

In [20]:
# Detecting last checkpoint.
last_checkpoint = None
if (
    os.path.isdir(training_args.output_dir)
    and training_args.do_train
    and not training_args.overwrite_output_dir
):
    last_checkpoint = get_last_checkpoint(training_args.output_dir)
    if last_checkpoint is None and len(os.listdir(training_args.output_dir)) > 0:
        raise ValueError(
            f"Output directory ({training_args.output_dir}) already exists and is not empty. "
            "Use --overwrite_output_dir to overcome."
        )
    elif last_checkpoint is not None:
        print(
            f"Checkpoint detected, resuming training at {last_checkpoint}. To avoid this behavior, change "
            "the `--output_dir` or add `--overwrite_output_dir` to train from scratch."
        )
f"""last_checkpoint is {last_checkpoint}"""

'last_checkpoint is None'

## prepare the model

In [21]:
model = Wav2Vec2ForCTC.from_pretrained("bakrianoo/sinai-voice-ar-stt")
model.to('cuda')

if model_args.freeze_feature_extractor:
    model.freeze_feature_extractor()

wer_metric = datasets.load_metric("wer")
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)
    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id
    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)
    # if logger.isEnabledFor(logging.DEBUG):
    for reference, predicted in zip(label_str[:5], pred_str[:5]):
        print(f'reference: "{reference}"')
        print(f'predicted: "{predicted}"')
            # if orthography.untransliterator is not None:
            #     logger.debug(f'reference (untransliterated): "{orthography.untransliterator(reference)}"')
            #     logger.debug(f'predicted (untransliterated): "{orthography.untransliterator(predicted)}"')
    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    return {"wer": wer}

# Data collator
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)
# Initialize our Trainer
trainer = CTCTrainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=processor.feature_extractor,
)
trainer.use_amp = False

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  import sys


## training

In [22]:
# Training
if training_args.do_train:
  if last_checkpoint is not None:
      checkpoint = last_checkpoint
  elif os.path.isdir(model_args.model_name_or_path):
      checkpoint = model_args.model_name_or_path
  else:
      checkpoint = None
  train_result = trainer.train(resume_from_checkpoint=checkpoint)
  trainer.save_model()

  # save the feature_extractor and the tokenizer
  if is_main_process(training_args.local_rank):
      processor.save_pretrained(training_args.output_dir)

  metrics = train_result.metrics
  max_train_samples = (
      data_args.max_train_samples
      if data_args.max_train_samples is not None
      else len(train_dataset)
  )
  metrics["train_samples"] = min(max_train_samples, len(train_dataset))

  trainer.log_metrics("train", metrics)
  trainer.save_metrics("train", metrics)
  trainer.save_state()

# Evaluation
results = {}
if training_args.do_eval:
  print("*** Evaluate ***")
  metrics = trainer.evaluate()
  max_val_samples = (
      data_args.max_val_samples
      if data_args.max_val_samples is not None
      else len(eval_dataset)
  )
  metrics["eval_samples"] = min(max_val_samples, len(eval_dataset))

  trainer.log_metrics("eval", metrics)
  trainer.save_metrics("eval", metrics)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2978
  Num Epochs = 10
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 8
  Total optimization steps = 1860
/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  "`as_target_processor` is deprecated and will be removed in v5 of Transformers. You can pro

Step,Training Loss,Validation Loss,Wer
200,0.383200,0.445711,0.383430
400,0.381600,0.496268,0.392674
600,0.365600,0.447275,0.437864
800,0.319500,0.381139,0.366655
1000,0.291000,0.379144,0.349880
1200,0.214200,0.333125,0.302978
1400,0.167300,0.294825,0.252311
1600,0.131800,0.264205,0.200274
1800,0.118600,0.259078,0.192742


***** Running Evaluation *****
  Num examples = 331
  Batch size = 8


reference: "فتلافته، فلاثت به      لعوة تضبح ضبح النهام"
predicted: "فتلا فته فلاثت به لعوة تضبح ضبح النهام"
reference: "آنست في الحي نارا      ليلا فبشرت أهلي"
predicted: "دا نست في الحينارا ليلا فمشرت ؤه"
reference: "وإن تنقضي العهد الذي كان بيننا      وتبدي بباقي ودك المتخلس"
predicted: "وإن تنقض العحد الذي كان بيننا  وتبدي بباق ودك المتخلس"
reference: "نام الخلي وبت الليل مرتفقا      أرعى النجوم عميدا مثبتا أرقا"
predicted: "نام الخلي وبط الليل مرتفقا أرع النجوم عميدا مثبةا أرقا"
reference: "إن العطية خير ما وجهتها      وحسبتها حمدا وأجرا واجبا"
predicted: "إن العطية خيب ما وجهتهاوحسبتها حد وأجرا وعجب"


Saving model checkpoint to /content/drive/MyDrive/KFUPM-Master/ICS606/Models/SinaiFineTuned/checkpoint-300
Configuration saved in /content/drive/MyDrive/KFUPM-Master/ICS606/Models/SinaiFineTuned/checkpoint-300/config.json
Model weights saved in /content/drive/MyDrive/KFUPM-Master/ICS606/Models/SinaiFineTuned/checkpoint-300/pytorch_model.bin
Feature extractor saved in /content/drive/MyDrive/KFUPM-Master/ICS606/Models/SinaiFineTuned/checkpoint-300/preprocessor_config.json
/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  "`as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your "
***** Running Evaluation *****
  Num examples = 331
  Batch size = 8


...

***** Running Evaluation *****
  Num examples = 331
  Batch size = 8


reference: "فتلافته، فلاثت به      لعوة تضبح ضبح النهام"
predicted: "فتلافته، فلاثت به    لعوة تضبح ضبح النهام"
reference: "آنست في الحي نارا      ليلا فبشرت أهلي"
predicted: "آنست في الحي نارا   ليلا فبشرت أهي"
reference: "وإن تنقضي العهد الذي كان بيننا      وتبدي بباقي ودك المتخلس"
predicted: "وإن تنقض العحد الذي كان بيننا   وتبدي بباق ودك المتخلس"
reference: "نام الخلي وبت الليل مرتفقا      أرعى النجوم عميدا مثبتا أرقا"
predicted: "نام الخلي وبت الليل مرتفقا   أرعى النجوم عميدا مثبة أرقا"
reference: "إن العطية خير ما وجهتها      وحسبتها حمدا وأجرا واجبا"
predicted: "إن لعطية خير ما وجهتها    وحسبتها حمدا وأجرا واجبا"


Saving model checkpoint to /content/drive/MyDrive/KFUPM-Master/ICS606/Models/SinaiFineTuned/checkpoint-1800
Configuration saved in /content/drive/MyDrive/KFUPM-Master/ICS606/Models/SinaiFineTuned/checkpoint-1800/config.json
Model weights saved in /content/drive/MyDrive/KFUPM-Master/ICS606/Models/SinaiFineTuned/checkpoint-1800/pytorch_model.bin
Feature extractor saved in /content/drive/MyDrive/KFUPM-Master/ICS606/Models/SinaiFineTuned/checkpoint-1800/preprocessor_config.json
Deleting older checkpoint [/content/drive/MyDrive/KFUPM-Master/ICS606/Models/SinaiFineTuned/checkpoint-1500] due to args.save_total_limit
/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  "`as_target_processor` is depreca

***** train metrics *****
  epoch                    =          10.0
  total_flos               = 16798743433GF
  train_loss               =        0.2991
  train_runtime            =    2:38:52.91
  train_samples            =          2978
  train_samples_per_second =         3.124
  train_steps_per_second   =         0.195
*** Evaluate ***


reference: "فتلافته، فلاثت به      لعوة تضبح ضبح النهام"
predicted: "فتلافته، فلاثت به   لعوة تضبح ضبح النهام"
reference: "آنست في الحي نارا      ليلا فبشرت أهلي"
predicted: "آنست في الحي نارا   ليلا فبشرت أهي"
reference: "وإن تنقضي العهد الذي كان بيننا      وتبدي بباقي ودك المتخلس"
predicted: "وإن تنقض العحد الذي كان بيننا   وتبدي بباق ودك المتخلس"
reference: "نام الخلي وبت الليل مرتفقا      أرعى النجوم عميدا مثبتا أرقا"
predicted: "نام الخلي وبت الليل مرتفقا   أرعى النجوم عميدا مثبة أرقا"
reference: "إن العطية خير ما وجهتها      وحسبتها حمدا وأجرا واجبا"
predicted: "إن لعطية خير ما وجهتها    وحسبتها حمدا وأجرا واجبا"
***** eval metrics *****
  epoch                   =       10.0
  eval_loss               =     0.2588
  eval_runtime            = 0:00:48.04
  eval_samples            =        331
  eval_samples_per_second =       6.89
  eval_steps_per_second   =      0.874
  eval_wer                =     0.1897


In [23]:
# use it if you face problems of checkpoints not being saved
from google.colab import drive
drive.flush_and_unmount()